In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
# Removed null rows
df = df.dropna()

In [13]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [20]:
# Independent features
X = df.drop('label', axis=1)
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
# Output feature
y = df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [23]:
# check whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [25]:
# vocabulary size
voc_size=5000

In [26]:
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
import nltk 
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# Since we removes null value rows indexes remain unaltered
messages.reset_index(inplace=True)

In [30]:
# Data Preprocessing 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  sentences = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  sentences = sentences.lower()
  words = sentences.split()

  words = [ps.stem(word) for word in words if not word in stopwords.words('english')]
  sentences = ' '.join(words)
  corpus.append(sentences)

In [36]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [38]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[869, 2632, 2571, 4871, 2717, 469, 3723, 1559, 1360, 370],
 [1255, 3104, 4893, 286, 772, 3541, 1752],
 [1133, 699, 1555, 354],
 [17, 4711, 4473, 1, 4097, 3082],
 [1154, 772, 4026, 158, 3350, 3914, 772, 3246, 1300, 3608],
 [4184,
  3079,
  584,
  83,
  1046,
  2496,
  385,
  4373,
  1592,
  1667,
  200,
  1142,
  373,
  1380,
  1752],
 [4739, 3881, 1108, 1828, 459, 1686, 4067, 3861, 224, 1795, 4527],
 [3288, 2599, 856, 2148, 2252, 383, 2496, 1481, 224, 1795, 4527],
 [88, 1389, 1334, 3238, 4049, 2800, 3984, 1556, 2496, 773],
 [494, 406, 2623, 4324, 4163, 4686, 1632, 1595],
 [1477, 875, 3579, 2062, 3370, 749, 219, 3021, 3673, 2397, 781],
 [1, 2342, 2717, 2800, 2496, 2252],
 [2163, 597, 2294, 1765, 1826, 4246, 930, 1190, 352],
 [4859, 3577, 1737, 3256, 1695, 4733, 3618, 224, 1795, 4527],
 [3598, 620, 3251, 4170, 1558, 224, 1795, 4527],
 [1063, 3438, 1876, 2607, 4035, 3626, 3546, 4752, 3027, 1082],
 [4678, 2646, 3104],
 [106, 350, 3271, 2384, 2496, 847, 3134, 1752],
 [2989, 3682, 4893, 467

# Embedding Representation

In [39]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr, maxlen=sent_length, padding='pre')
embedded_docs

array([[   0,    0,    0, ..., 1559, 1360,  370],
       [   0,    0,    0, ...,  772, 3541, 1752],
       [   0,    0,    0, ...,  699, 1555,  354],
       ...,
       [   0,    0,    0, ...,  224, 1795, 4527],
       [   0,    0,    0, ...,  100,  506, 4509],
       [   0,    0,    0, ..., 3018, 3809, 4719]], dtype=int32)

In [41]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42 )

In [45]:
# model training
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
383/383 [==============================] - 12s 24ms/step - loss: 0.2604 - accuracy: 0.8831 - val_loss: 0.2013 - val_accuracy: 0.9190
Epoch 2/10
383/383 [==============================] - 7s 19ms/step - loss: 0.1327 - accuracy: 0.9495 - val_loss: 0.2045 - val_accuracy: 0.9200
Epoch 3/10
383/383 [==============================] - 8s 20ms/step - loss: 0.0892 - accuracy: 0.9668 - val_loss: 0.2422 - val_accuracy: 0.9185
Epoch 4/10
383/383 [==============================] - 7s 20ms/step - loss: 0.0536 - accuracy: 0.9822 - val_loss: 0.2769 - val_accuracy: 0.9183
Epoch 5/10
383/383 [==============================] - 7s 19ms/step - loss: 0.0330 - accuracy: 0.9898 - val_loss: 0.3079 - val_accuracy: 0.9157
Epoch 6/10
383/383 [==============================] - 7s 20ms/step - loss: 0.0230 - accuracy: 0.9922 - val_loss: 0.3986 - val_accuracy: 0.9150
Epoch 7/10
383/383 [==============================] - 9s 24ms/step - loss: 0.0173 - accuracy: 0.9939 - val_loss: 0.3953 - val_accuracy: 0.913

In [46]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>=0.5,1,0)

In [47]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 255, 2361]])

In [48]:
print(accuracy_score(y_test,y_pred))

0.907705053852527
